# Cleaning the data:

In [69]:
# Import required libraries
import pandas as pd

# Collection of functions for scientific and publication-ready visualization
import matplotlib.pyplot as plt 

# Visualization library based on matplotlib, provides interface for drawing attractive statistical graphics
# import seaborn as sns  # Install seaborn in myEnvPy3 environment

ModuleNotFoundError: No module named 'seaborn'

In [74]:
# Load the split data
splitData = pd.read_csv('./data/splitKinectData.csv')

In [71]:
# To describe the data with count, mean, std, min, etc..

# splitData.describe()

In [3]:
# To see the top part of data

# splitData.head()

In [25]:
# To see the bottom part of the data

# splitData.tail()

In [2]:
# To see a sample of whole data

# splitData.sample(5)

In [82]:
# To check datatype and other info of each column
# By lookin at total number of entries and number of non-null objects we can check 
# if there are any null values or missing data

# splitData.info()

In [79]:
# Mapping 'face_engaged': no = 0; yes = 1
# With yes and no: face_looking away, face_engaged

yesNoList = ['face_engaged', 'face_lookingaway']

for i in range(len(yesNoList)):
    splitData[yesNoList[i]] = splitData[yesNoList[i]].map( {'no': 0, 'yes': 1, 'unknown': 0} ).astype(int)
    
# Mapping 'face_glasses': unknown & no = 0; yes = 1 ###Commented and created a loop###
# splitData['face_glasses'] = splitData['face_glasses'].map( {'no': 0, 'yes': 1, 'unknown': 0} ).astype(int)

# unknown, no yes: face_happy, face_lefteyeclosed, face_mouthmoved, face_mouthopen, face_righteyeclosed
unknownYesNoList = ['face_glasses', 'face_happy', 'face_lefteyeclosed', 'face_mouthmoved',
                    'face_mouthopen', 'face_righteyeclosed']
for i in range(len(unknownYesNoList)):
    splitData[unknownYesNoList[i]] = splitData[unknownYesNoList[i]].map( {'no': 0, 'yes': 1, 'unknown': 0} ).astype(int)

In [60]:
# Dropping face size since it's a column of zeros
# splitData.drop(columns=['face_size'])
# splitData
# splitData.describe()

In [59]:
# Dropping First column of indexes since it is redundant
# splitData.drop('Unnamed: 0', axis=1, inplace=True)
# splitData.sample()

In [5]:
# Visualizing individual features
# splitData['pedes_posX'].plot()

## Passing the data to a unsupervised learning algorithm:

In [15]:
# import required libraries
from sklearn import preprocessing
import numpy as np

In [16]:
# Scaling features to a range
# initialize Min-max scaler
minMaxScaler = preprocessing.MinMaxScaler()

# copy df to be compared later
unprocessData = splitData

In [17]:
# Normalizing Data
# dataNormalized = preprocessing.normalize(unprocessData, norm='l2')

# MinMax Scaling Unprocessed Data
dataMinMax = minMaxScaler.fit_transform(unprocessData)

# MinMax Scaling Normalized Data
# dataMinMax = minMaxScaler.fit_transform(dataNormalized)

To Check if k-value for PCA retains 99% of variance:

In [88]:
# Create Covariance Matrix
numTrainEg = dataMinMax.shape
covarMatrix = (1/numTrainEg[0])*(np.matmul(dataMinMax.T, dataMinMax))
covarMatrix.shape
# Get svd of Covariance Matrix
u, s, v = np.linalg.svd(covarMatrix, full_matrices=True)
# Checking how much variance we have retained
sum(s[:2])/sum(s)

0.8834624645663419

In [97]:
# Try k-means clustering
# import clusters
from sklearn import cluster

# initialize k-means
k_means = cluster.KMeans(n_clusters=3)
k_means.fit(dataMinMax)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [115]:
# labels = k_means.labels_
# centers = k_means.cluster_centers_
# help(k_means)

In [18]:
from sklearn.decomposition import PCA
import numpy as np

kmeansReduced = cluster.KMeans(init='k-means++', n_clusters=3, n_init=10)

# reduced_data = PCA(n_components=2).fit_transform(dataNormalized)
reduced_data = PCA(n_components=2).fit_transform(dataMinMax)

kmeansReduced.fit(reduced_data)

# Step size of the mesh. Decrease to increase the quality of the VQ.
h = .02     # point in the mesh [x_min, x_max]x[y_min, y_max].

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = kmeansReduced.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
fig = plt.figure(1)
plt.clf()
plt.imshow(Z, interpolation='nearest',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap=plt.cm.Paired,
           aspect='auto', origin='lower')

plt.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', markersize=2)
# Plot the centroids as a white X
centroids = kmeansReduced.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='x', s=169, linewidths=3,
            color='w', zorder=10)
# plt.title('K-means clustering on Normalized Data\n'
#           'Centroids are marked with white cross')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())

# Make a selection
# plt.savefig('kMeansMinMaxPCA.eps', format='eps', dpi=100)
# plt.savefig('kMeansNormalizedPCA.eps', format='eps', dpi=100)
# plt.savefig('kMeansNormMinMaxPCA.eps', format='eps', dpi=100)
plt.show()


NameError: name 'cluster' is not defined

In [87]:
# Only Testing PCA
from sklearn.decomposition import PCA
import numpy as np

# kmeansReduced = cluster.KMeans(init='k-means++', n_clusters=3, n_init=10)

# reduced_data = PCA(n_components=2).fit_transform(dataNormalized)
reduced_data = PCA(n_components=2)
reduced_data.fit_transform(dataMinMax)
print(reduced_data.explained_variance_ratio_.cumsum())

[0.3994836  0.58945631]


In [5]:
tomoyaData = pd.read_csv('tomoyoLeapCat_100_0.csv')

In [6]:
tomoyaData.sample(5)

,Unnamed: 0,Time,thumbProximal_L_X,thumbProximal_L_Y,thumbProximal_L_Z,thumbDistal_L_X,thumbDistal_L_Y,thumbDistal_L_Z,thumbEF_L_X,thumbEF_L_Y,...,armFrontRadius_R_Z,armFrontUnla_R_X,armFrontUnla_R_Y,armFrontUnla_R_Z,armBackLateral_R_X,armBackLateral_R_Y,armBackLateral_R_Z,armBackMedial_R_X,armBackMedial_R_Y,armBackMedial_R_Z
13,13,13,-0.154221,-0.009801,0.351608,-0.154221,-0.009801,0.351608,-0.125315,0.021736,...,0.320950,0.071754,-0.031825,0.354184,0.191447,-0.170060,0.207293,0.208151,-0.180164,0.240527
31,31,31,-0.154094,-0.009933,0.351446,-0.154094,-0.009933,0.351446,-0.125453,0.021754,...,0.307211,0.077601,-0.025145,0.341086,0.193232,-0.174480,0.202013,0.209081,-0.183807,0.235887
64,64,64,-0.154779,-0.009379,0.351216,-0.154779,-0.009379,0.351216,-0.126826,0.022690,...,0.296780,0.208874,-0.014308,0.330060,0.160117,-0.214562,0.218106,0.173198,-0.228937,0.251386
77,77,77,-0.154391,-0.009295,0.350695,-0.154391,-0.009295,0.350695,-0.126037,0.022484,...,0.290330,0.178661,-0.004194,0.325022,0.171782,-0.210765,0.214251,0.183721,-0.222570,0.248943
70,70,70,-0.154457,-0.009486,0.351018,-0.154457,-0.009486,0.351018,-0.126409,0.022589,...,0.295726,0.199935,-0.009963,0.330075,0.164159,-0.211359,0.215317,0.174564,-0.225407,0.249667


Audio Data Test

# Deep Learning

In [106]:
# Drop last n rows
n = 100
splitData.drop(splitData.tail(n).index,inplace=True)

In [115]:
# Seperate into three new df
approachDf = splitData.iloc[:300, :]
interactDf = splitData.iloc[300:600, :]
leaveDf = splitData.iloc[600:, :]

In [122]:
approachDf.to_csv('approachDf' + '.csv')
interactDf.to_csv('interactDf' + '.csv')
leaveDf.to_csv('leaveDf' + '.csv')


In [195]:
# Tomoya part
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.contrib import rnn
from keras.utils import to_categorical
from itertools import chain
import matplotlib.pyplot as plt
import csv
import seaborn as sn
from sklearn.metrics import confusion_matrix

In [94]:
def labels_2_one_hots(labels=''):
    words = []
    new_word_id = 0
    dictionary = {}
    labels = labels[:, np.newaxis]
    # sort by alphabetical order
    labelsSorted = np.sort(labels, axis=0)

    for word in labelsSorted:
        if word[0] not in dictionary:
            dictionary[word[0]] = new_word_id
            new_word_id += 1


            for word in labels:
        words.append(dictionary[word[0]])

    one_hots = to_categorical(words)
    dictionary_inv = {dictionary[k]: k for k in dictionary}

    return dictionary, one_hots, dictionary_inv

In [192]:
def get_train_test_data():

    # Get current working directory of process
    cwd = os.getcwd()
    path = cwd + '/data' + '/deepLearning'
    files = []
    for file in os.listdir(path):
        if file.endswith(".csv"):
            files.append(file)

    files = np.array(files)
    labelsOutput = []
    InputAll = []

    for i in range(0, int(files.shape[0])):
        dfInput = pd.read_csv(path + '/' + files[i], sep=',')
    #     dfInput.dropna(how='any') # Drop missing values
        dfInput.drop(['Unnamed: 0'], axis=1, inplace=True)
        print(files[i] + ' open')
        label = str.lower(files[i][:-4]) # Remove '.csv'
        InputNames = dfInput.columns

        Input = dfInput.as_matrix() # Convert the frame to its Numpy-array representation
        nCut = int(Input.shape[0] / 100)  

        # Convert from Rows, Columns(2D) to nCut, 100, Columns(3D)
        Input = np.array(np.split(Input, nCut, axis=0))
        # Transpose to 100, Columns, nCut
        Input = Input.transpose([1, 2, 0])

        labelsOutput.append([label for j in range(0, nCut)])

        if InputAll == []:
            InputAll = Input
        else:
            InputAll = np.concatenate([InputAll, Input], axis=2)

    labelsOutput = np.array(list(chain.from_iterable(labelsOutput))) # chain('ABC', 'DEF') --> A B C D E F

    p = np.random.permutation(InputAll.shape[2])

    InputAll = InputAll[:, :, p]

    labelsOutput = labelsOutput[p]

    dictionary, one_hots, dictionary_inv = labels_2_one_hots(labelsOutput)
    
    return InputAll, dictionary, one_hots, InputNames, dictionary_inv

In [189]:
def standardize_data(data):
    std = []
    mean = []
    dataOut = data.copy()

    std.append(np.std(data, ddof=1))
    mean.append(np.mean(data))
    dataOut = (data - mean) / std

    dataOut[np.isnan(dataOut)] = 0

    return dataOut, mean, std


In [193]:
InputAll, dictionary, one_hots, InputNames, dictionary_inv = get_train_test_data()

leaveDf.csv open
approachDf.csv open
interactDf.csv open


/home/sp/anaconda3/envs/myEnvPy3/lib/python3.6/site-packages/ipykernel/__main__.py:33: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


In [204]:
class RNN:
    def __init__(RNN):

#         RNN.hm_epochs = 10000
        RNN.hm_epochs = 3

    def set_Data(RNN, Input, Output, InputNames, LabelsDict, dictionary_inv):

        RNN.InputNames = InputNames
        RNN.OutputNames = sorted(LabelsDict, key = LabelsDict.get, reverse = False)
        RNN.dictionary = LabelsDict
        RNN.dictionary_inv = dictionary_inv

        RNN.n_samples = Input.shape[2]
        RNN.training_size = int(0.8 * RNN.n_samples)
        RNN.testing_size = int(0.2 * RNN.n_samples)

        Input_N, RNN.meanInput, RNN.stdInput = standardize_data(Input)

        RNN.train_x = np.array(Input[:, :, :RNN.training_size]).astype('float32')
        RNN.train_x = np.transpose(RNN.train_x, [2, 0, 1])
        RNN.train_y = np.array(Output[:RNN.training_size])

        RNN.test_x = np.array(Input[:, :, -RNN.testing_size:]).astype('float32')
        RNN.test_x = np.transpose(RNN.test_x, [2, 0, 1])
        RNN.test_y = np.array(Output[-RNN.testing_size:])

        RNN.train_x_N = np.array(Input_N[:, :, :RNN.training_size]).astype('float32')
        RNN.train_x_N = np.transpose(RNN.train_x_N, [2, 0, 1])
        RNN.test_x_N = np.array(Input_N[:, :, -RNN.testing_size:]).astype('float32')
        RNN.test_x_N = np.transpose(RNN.test_x_N, [2, 0, 1])

    def train_neural_network(RNN, InputAll, one_hots):

        errorHistory =[]

        tf.reset_default_graph()

        sess = tf.Session()

        frames = 100
#         n_Joint = 162
        n_Joint = 58

        CellSize = 10
#         OutputSize = 20
        OutputSize = 3

        RNN.x = tf.placeholder('float32',
                           [None, frames, n_Joint])  # TensorShape([Dimension(None), Dimension(28), Dimension(28)])
        RNN.y = tf.placeholder('float32', [None, OutputSize])


        layer = {'weights': tf.Variable(tf.random_normal([CellSize, OutputSize])),
                 'biases': tf.Variable(tf.random_normal([OutputSize]))}

        x = tf.transpose(RNN.x, [1, 0, 2])  # TensorShape([Dimension(28), Dimension(None), Dimension(28)])
        x = tf.reshape(x, [-1, n_Joint])
        x = tf.split(x, frames, 0)  # len(x) = 100

        lstm_cell = rnn.BasicRNNCell(CellSize)

        outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

        RNN.y_ = tf.matmul(outputs[-1], layer['weights']) + layer['biases']

        learningRate = 0.01
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=RNN.y_, labels=RNN.y))
        optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(cost)

        correct = tf.equal(tf.argmax(RNN.y_, 1), tf.argmax(RNN.y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

        sess.run(tf.global_variables_initializer())

        for epoch in range(RNN.hm_epochs):

            _, c = sess.run([optimizer, cost], feed_dict={RNN.x:RNN.train_x_N , RNN.y: RNN.train_y})
            errorHistory.append(c)
            if epoch > 100:
                if epoch % 100 == 0:
                    if abs(errorHistory[epoch] - errorHistory[epoch-100]) < 0.01:
                        break
            print('Epoch', epoch, 'completed out of', RNN.hm_epochs, 'loss:', c)

        test_data = {RNN.x: RNN.test_x_N, RNN.y: RNN.test_y}

        RNN.accuracy = sess.run(accuracy, feed_dict = test_data)

        input_data = {RNN.x: RNN.test_x_N}

        RNN.output_rnn = sess.run(RNN.y_, feed_dict=input_data)

        RNN.output_probabilities = tf.nn.softmax(RNN.output_rnn)
        RNN.output_probabilities = sess.run(RNN.output_probabilities)
        RNN.output_probabilities = np.array(RNN.output_probabilities)

        RNN.output_OneHot = (RNN.output_probabilities == RNN.output_probabilities.max(axis = 1, keepdims = True)).astype(int)

        RNN.output_label = []

        for i in range(0, len(RNN.dictionary)):
            RNN.output_label_temp = RNN.dictionary_inv.get(np.argmax(RNN.output_OneHot[i]))
            RNN.output_label = np.append(RNN.output_label, RNN.output_label_temp)
            # RNN.output_label = np.core.defchararray.add(RNN.output_label, RNN.output_label_temp)

        print('Accuracy:', RNN.accuracy)
        plt.figure()
        plt.plot(errorHistory)
        plt.title('Loss_value')
        plt.savefig('Loss' + RNN.accuracy + '.png')

        f = open('loss' + RNN.accuracy + '.csv', 'w')
        writer = csv.writer(f, lineterminator='/n')
        writer.writerow(errorHistory)
        f.close()

    def draw_confusion_matrix(RNN):

        RNN.test_label = []

        for i in range(0, len(RNN.dictionary)):
            RNN.test_label_temp = RNN.dictionary_inv.get(np.argmax(RNN.test_y[i]))
            RNN.test_label = np.append(RNN.test_label, RNN.test_label_temp)

        labels = sorted(list(set(RNN.test_label)))
        cmx_data = confusion_matrix(RNN.test_label, RNN.output_label, labels=labels)
        with np.errstate(divide='ignore', invalid='ignore'):
            cmx_data_N = np.true_divide(cmx_data, cmx_data.astype(np.float).sum(axis=0))
            cmx_data_N = np.nan_to_num(cmx_data_N)

        df_cmx = pd.DataFrame(cmx_data.T, index=labels, columns=labels)
        df_cmx_N = pd.DataFrame(cmx_data_N.T, index=labels, columns=labels)

        np.savetxt('100.txt', cmx_data_N)

        # plt.figure()
        # sn.heatmap(df_cmx, vmax=1, vmin=0, annot=True, robust=True)
        # plt.show()

        plt.figure(figsize=(30,30))
        sn.heatmap(df_cmx_N, vmax=1, vmin=0, annot=True)
        plt.show()
        plt.savefig('cmx.png')

In [205]:
RNN = RNN()
RNN.set_Data(Input=InputAll, Output=one_hots, InputNames=InputNames, LabelsDict=dictionary, dictionary_inv=dictionary_inv)
print('-----done.')
RNN.train_neural_network(InputAll, one_hots)
RNN.draw_confusion_matrix()

-----done.
Epoch 0 completed out of 3 loss: 4.799125
Epoch 1 completed out of 3 loss: 4.1926184
Epoch 2 completed out of 3 loss: 3.5916579


IndexError: index 1 is out of bounds for axis 0 with size 1